In [2]:
import random as rd
import copy

def choice_wo(l, ex):
    la=copy.deepcopy(l)
    for a in ex:
        la.remove(a)
    return rd.choice(la)
def list_wo(la,ex):
    l=copy.deepcopy(la)
    a=[]
    for x in l:
        accept=True
        for exp in ex:
            if x==exp:
                accept=False
        if accept==True:
            a.append(x)
    return a
def mergeify(l):
    resp=[]
    for x in range(len(l[0])):
        item=[]
        for y in l:
            if type(y[x])==list:
                for z in y[x]:
                    item.append(z)
            else:
                item.append(y[x])
        resp.append(item)
    return resp
def allorders(l):
    if len(l)==1:
        return [l]
    resp=[]
    for x in l:
        rest=list_wo(l,[x])
        for y in allorders(rest):
            resp.append([x]+y)
    return(resp)
def remfrom(l, it):
    for x in l:
        if x==it:
            return l.remove(it)
    return l
def addcount(l):
    ans=[]
    for x in range(len(l)):
        l[x].append(x)
        ans.append(l[x])
    return ans
def owner(answer,trait):
    for x in answer:
        for y in x:
            if y==trait:
                return x
    return None

def forall(pre,l):
    ans=[]
    if l==[] or l==None:
        return pre
    for x in range(len(l[0])):
        if pre!=[]:
            newpre=mergeify([pre,l[0][x]])
        else:
            newpre=l[0][x]
        ans+=(forall(newpre,l[1:]))
    return (ans)
def clump(l,size):
    ans=[]
    for x in range(int(len(l)/size)):
        clump=[]
        for y in l[size*x:size*(x+1)]:
            clump.append(y)
        ans.append(clump)
    return ans

def combos(pre,l):
    ords=[allorders(x) for x in l]
    a=forall(pre, ords)
    batchsize=len(l[0])
    return clump(a,batchsize)

In [151]:
typ_options=['equal', 'distin', 'eq_or_1s']

class puzzle_base:
    def __init__(self,rs,ans):
        self.keys=[(x,rs[x]) for x in range(len(rs))]+[(len(rs),'position')]
        self.dict={y:x for x,y in self.keys}
        self.chars=copy.deepcopy(ans)
        for x in range(len(ans)):
            ans[x].append(x)
        self.all_ans=ans
        self.options=[]
        for x in self.keys:
            if x[1]=='position':
                break
            portion=[]
            for y in self.chars:
                portion.append(y[x[0]])
            self.options.append(portion)
        self.clues=[]
        
    def all_except(self,l):
        others=copy.deepcopy(self.all_ans)
        for char in l:
            others.remove(char)
        return others
    
    def evaluate(self):
        poss=0
        count=0
        for a in allorders(self.options[0]):
            count+=1
            print('{0}/24'.format(count))
            for z in combos(a,self.options[1:]):
                ans=addcount(z)
                #evaluate clues
                match=True
                for clue in self.clues:
                    if clue[0]=='equal' and owner(ans,clue[1][0])!=owner(ans,clue[1][1]):
                        match=False
                    elif clue[0]=='distin' and owner(ans,clue[1][0])==owner(ans,clue[1][1]):
                        match=False
                if match==True:
                    poss+=1
                    if poss>1:
                         return False
        return True

    def clue(self, typ, char):
        if typ=='equal':
            p1=rd.choice(char)
            p2=choice_wo(char,[p1])
            specs=[p1,p2]
        elif typ=='distin':
            others=self.all_except([char])
            p1_t=rd.choice(self.keys)
            p2_t=choice_wo(self.keys,[p1_t])
            p1=char[p1_t[0]]
            p2=rd.choice(others)[p2_t[0]]
            specs=[p1,p2]
        elif typ=='eq_or_1s':
            other=rd.choice(self.all_except([char]))
            p1_t=rd.choice(self.keys)
            p2_t=choice_wo(self.keys,[p1_t])
            pid=char[p1_t[0]]
            p1=char[p2_t[0]]
            p2=other[p2_t[0]]
            specs=[pid,p1,p2]
        clue=(typ,specs)
        self.clues.append(clue)
        return clue
    
    def filter_clues(self):
        probs=[]
        for x in range(len(self.clues)):
            for y in range(len(self.clues)):
                if self.clues[x]==self.clues[y] and x>y:
                    probs.append(self.clues[x])
                    print('found one')
                elif self.clues[x][0]=='equal' and self.clues[y][0]=='equal' and self.clues[x][1][0]==self.clues[y][1][1] and self.clues[x][1][1]==self.clues[y][1][0] and x>y:
                    print(self.clues[x])
                    print(self.clues[y])
                    probs.append(self.clues[x])
        for prob in probs:
            remfrom(self.clues,prob)

In [152]:
Names=['Willow', 'Lisbet', 'Cameron', 'Mariette', 'Annie', 'Luna', 'Celia']
Scents=['pine', 'lavender', 'lemon', 'soap', 'fish']
Shoes=['hiking shoes', 'worn sneakers', 'winter boots', 'rainboots', 'trail-runners']
BirthMons=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
Coats=['hoodie', 'sweater', 'raincoat', 'down jacket', 'sports coat', 'denim jacket']
Colors=['navy blue', 'pale blue', 'white', 'grey', 'black', 'beige']

sample=puzzle_base(['name', 'coat', 'color', 'scent', 'hobby'], [['Willow', 'raincoat', 'white', 'pine', 'piano'], 
              ['Cameron', 'hoodie', 'navy blue', 'cinnamon', 'reading'], ['June', 'fleece', 'grey', 'lavender', 'singing'], 
              ['Sam', 'fur coat', 'tan', 'fish', 'sewing']])
#owner(sample.all_ans,'sewing')

In [170]:
#sample.clue(rd.choice(typ_options),rd.choice(sample.all_ans))
sample.clue('equal',rd.choice(sample.all_ans))
sample.clues

found one
('equal', ['hoodie', 'cinnamon'])
('equal', ['cinnamon', 'hoodie'])


[('equal', ['raincoat', 0]),
 ('equal', [0, 'Willow']),
 ('equal', ['cinnamon', 'hoodie']),
 ('equal', ['navy blue', 1]),
 ('equal', ['white', 'pine']),
 ('equal', ['white', 'raincoat']),
 ('equal', [2, 'singing']),
 ('equal', ['white', 0]),
 ('equal', ['lavender', 'fleece']),
 ('equal', ['cinnamon', 1]),
 ('equal', ['white', 'piano']),
 ('equal', [3, 'Sam']),
 ('equal', ['hoodie', 1]),
 ('equal', ['Sam', 'sewing']),
 ('equal', ['fish', 'Sam']),
 ('equal', [2, 'fleece'])]

In [5]:
a=[11,2,0]
b=['a','c','b']
c=['cat', 'dog','bird']
for y in combos([],[a,b,c]):
    print(y)

[[11, 'a', 'cat'], [2, 'c', 'dog'], [0, 'b', 'bird']]
[[11, 'a', 'cat'], [2, 'c', 'bird'], [0, 'b', 'dog']]
[[11, 'a', 'dog'], [2, 'c', 'cat'], [0, 'b', 'bird']]
[[11, 'a', 'dog'], [2, 'c', 'bird'], [0, 'b', 'cat']]
[[11, 'a', 'bird'], [2, 'c', 'cat'], [0, 'b', 'dog']]
[[11, 'a', 'bird'], [2, 'c', 'dog'], [0, 'b', 'cat']]
[[11, 'a', 'cat'], [2, 'b', 'dog'], [0, 'c', 'bird']]
[[11, 'a', 'cat'], [2, 'b', 'bird'], [0, 'c', 'dog']]
[[11, 'a', 'dog'], [2, 'b', 'cat'], [0, 'c', 'bird']]
[[11, 'a', 'dog'], [2, 'b', 'bird'], [0, 'c', 'cat']]
[[11, 'a', 'bird'], [2, 'b', 'cat'], [0, 'c', 'dog']]
[[11, 'a', 'bird'], [2, 'b', 'dog'], [0, 'c', 'cat']]
[[11, 'c', 'cat'], [2, 'a', 'dog'], [0, 'b', 'bird']]
[[11, 'c', 'cat'], [2, 'a', 'bird'], [0, 'b', 'dog']]
[[11, 'c', 'dog'], [2, 'a', 'cat'], [0, 'b', 'bird']]
[[11, 'c', 'dog'], [2, 'a', 'bird'], [0, 'b', 'cat']]
[[11, 'c', 'bird'], [2, 'a', 'cat'], [0, 'b', 'dog']]
[[11, 'c', 'bird'], [2, 'a', 'dog'], [0, 'b', 'cat']]
[[11, 'c', 'cat'], [2, 'b', 